# Testing The API

## Imports

In [161]:
from typing import List
from pydantic import BaseModel
from enum import Enum
from fastapi import FastAPI
from neo4j import GraphDatabase
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import Neo4jVector
from langchain.embeddings import HuggingFaceInstructEmbeddings, HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA  # Q&A retrieval system.
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from operator import itemgetter
import uvicorn
import os
import langchain
from datetime import datetime
from pydantic import BaseModel


## Data Model

In [212]:
## DATA MODEL =============================================
class FeedbackEnum(str, Enum):
    """ 
        A Pydantic model representing feedback. 
    """
    like = 1
    neutral = 0
    dislike = -1

class Prompt(BaseModel):
    """
    A Pydantic model representing a LLM prompt
    """
    prompt: str

class Question(BaseModel):
    """
    A Pydantic model representing a search question. 
    """
    search_string: str

class ChunkMetadata(BaseModel):
    """
    A Pydantic model representing metadata.
    """
    chunk_size: int
    qa_embedding_model: str
    retrieval_embedding_model: str
    chunk_order: int
    chunk_overlap: int
    chunk_id: int

class Document(BaseModel):
    """
    A Pydantic model representing a document.
    Wrapper around langchains langchain.schema.document.Document.
    """
    page_content: str
    metadata: ChunkMetadata

class Answer(BaseModel):
    """
    A Pydantic model representing an answer to a question.
    """
    context: List[Document]
    llm_prompt: str
    llm_answer: str
    language: str
    score: FeedbackEnum


## Constants

In [158]:
BASE_PROMPT_TEMPLATE = """
ANSWER THE QUESTION BASED ONLY ON THE FOLLOWING CONTEXT:
{context}

QUESTION: 
{question}

ANSWER IN THE FOLLOWING LANGUAGE: 
{language}

CLEARLY STATE IF THE ANSWER CANNOT BE FOUND IN THE CONTEXT ABOVE.
IF THE ANSWER CAN BE FOUND, REFERENCE THE CONTEXT. 
"""
# Define the configuration
RETRIEVER_SEARCH_CONFIG = {
    # "similarity" (default), "mmr", or "similarity_score_threshold".
    'search_type': 'similarity', 
    'search_kwargs': {
        # Amount of documents to return (default: 4).
        'k': 5, 
        # Amount of documents to pass to the MMR algorithm 
        # # (default: 20).
        'fetch_k': 50, 
        # Minimum relevance threshold for similarity_score_threshold.
        'score_threshold': 0, 
        # Diversity of results returned by MMR; 
        # # 1 for minimum diversity and 0 for maximum (default: 0.5).
        'lambda_mult': 0.25, 
        # Filter by document metadata.
        'filter': {'chunk_size': 500}
    }
}


## Global Variables

In [180]:
#### GLOBAL VARIABLES =================
langchain.verbose = False
langchain.debug = False
load_dotenv() 

# Instructor Embeddings

model_kwargs = {
    'device': 'cpu'
}

encode_kwargs = {
    # 'normalize_embeddings': True,
    'show_progress_bar': False
}

instructor_model = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-xl", 
    cache_folder='./models/model_cache_xl',
    embed_instruction="Represent the Medical question for retrieving supporting paragraphs: ",
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

robbert_model = HuggingFaceEmbeddings(
    model_name="jegorkitskerkin/robbert-v2-dutch-base-mqa-finetuned", 
    cache_folder='./models/robbert-v2-dutch-base-mqa-finetuned',
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# Graph from existing graph
neo4j_qa_graph = Neo4jVector.from_existing_index(
    embedding=robbert_model,
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USER"),
    password=os.getenv("NEO4J_PASSWORD"),
    index_name="vi_chunk_qa_embedding_cosine",
    keyword_index_name="fts_Chunk_text",
    search_type="hybrid",
)

neo4j_retrieval_graph = Neo4jVector.from_existing_index(
    embedding=instructor_model,
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USER"),
    password=os.getenv("NEO4J_PASSWORD"),
    index_name="vi_chunk_retrieval_embedding_cosine",
    keyword_index_name="fts_Chunk_text",
    search_type="hybrid",
)


# Neo4j driver
driver = GraphDatabase.driver(
    os.getenv("NEO4J_URI"), 
    auth=(
        os.getenv("NEO4J_USER"), 
        os.getenv("NEO4J_PASSWORD")
    )
)

# Langchain retriever
neo4j_qa_retriever = neo4j_qa_graph.as_retriever(**RETRIEVER_SEARCH_CONFIG)
neo4j_retrieval_retriever = neo4j_retrieval_graph.as_retriever(**RETRIEVER_SEARCH_CONFIG)

load INSTRUCTOR_Transformer
max_seq_length  512


In [220]:
from langchain.docstore.document import Document

def docs_to_str(l: List[langchain.schema.document.Document]) -> str:
    page_content: str = ""
    for i, d in enumerate(l): 
        # Page Content
        page_content += (
            (25 * "=") + 
            (f" Document {i+1} ") + 
            (25 * "=") + 
            '\n'
        )
        page_content += (
            # (25 * "=" ) + 
            (f"# Document {i+1} Metadata ") + 
            # (5 * "=" ) + 
            '\n'
        )
        # Metadata 
        for k, v in d.metadata.items():
            page_content += (f"- {k}: {v} \n")
        
        page_content += (
            # (25 * "=" ) + 
            (f"# Document {i+1}'s Content") + 
            # (5 * "=" ) + 
            '\n'
        )
        page_content += d.page_content
        page_content += '\n'
    return page_content


def chunk_paths_to_docs(chunk_paths: List[object]) -> List[Document]:
    """
        For now, this matches chunk_paths of type (Chunk)-rel-(WebPage)-()
    """
    # This returns paths, that we can turn into LangChain documents somehow. 
    path_docs: List[Document] = []

    # One result for every chunk (see above)
    for p in chunk_paths:
        chunk_path_str = '' 
        chunk_node = p['rel'][0] 
        chunk_text = chunk_node.get('text')
        
        # Build up metadata of Document object manually
        doc_meta = {
            'chunk_size': chunk_node.get('chunk_size'),
            'qa_embedding_model': chunk_node.get('qa_embedding_model'),
            'retrieval_embedding_model': chunk_node.get('retrieval_embedding_model'),
            'chunk_order': chunk_node.get('chunk_order'),
            'chunk_overlap': chunk_node.get('chunk_overlap'),
            'chunk_id': chunk_node.get('chunk_id'),
        }
        # Traverse path for metadata
        for i, o in enumerate(p['rel']): 
            # Create path representation
            if type(o) == dict:
                chunk_path_str += f'(Node)'
            elif type(o) == str:
                chunk_path_str += f'<-{o}-'
            
            # WebPage node data
            if i == 2:
                doc_meta['webpage_scrape_dt'] = o.get('scrape_dt')
                doc_meta['webpage_url'] = o.get('url')
                doc_meta['webpage_title'] = o.get('title')
            # Catalog node data
            elif i == 4: 
                doc_meta['catalog_url'] = o.get('url')
        
        # Add path structure as metadata
        doc_meta['path_context'] = chunk_path_str   
        
        # Extract metadata from traversal 
        path_docs.append(
            Document(
                page_content=chunk_text, 
                metadata=doc_meta
            )
        )
    return path_docs

def get_neo4j_node_paths( 
        db: Neo4jVector, 
        question: str, 
        embedding_model: object,
        embedding_index: str, 
        limit: int = 5,) -> List[langchain.schema.document.Document]:
    vec_chunk_paths = db.query(
        f"""
            CALL db.index.vector.queryNodes(
                '{embedding_index}', 
                {limit},
                {embedding_model.embed_query(question)}
            ) 
            YIELD node, score
            WITH node, score
            ORDER BY score DESCENDING
            MATCH rel=(node:Chunk)<-[:HAS_CHUNK]-(:WebPage)<-[:HAS_WEBPAGE]-(:Catalog)
            RETURN DISTINCT rel
        """
    )
    return vec_chunk_paths


AttributeError: module 'langchain.docstore' has no attribute 'Document'

In [216]:

neo4j_paths = get_neo4j_node_paths(
    db=neo4j_qa_graph, 
    question="Wat is een virus?", 
    embedding_model=robbert_model,
    embedding_index="vi_chunk_qa_embedding_cosine", 
    limit=5,
)

neo4j_docs = chunk_paths_to_docs(neo4j_paths)

In [218]:
neo4j_docs[0]

Document(page_content='### Infectie\n\n\nBij een infectie zijn ziekteverwekkende micro-organismen aanwezig in het lichaam die zich kunnen vermenigvuldigen waardoor ontsteking of ziekte ontstaat. Een infectie kan optreden na een operatie of kan andere oorzaken hebben. Dat vertaalt zich meestal in koorts.  \n\n\n\n\n## Ontslag', metadata=ChunkMetadata(chunk_size=500, qa_embedding_model='egorkitskerkin/robbert-v2-dutch-base-mqa-finetuned', retrieval_embedding_model='hkunlp/instructor-xl with instruction: Represent the Medical paragraph for retrieval', chunk_order=4377, chunk_overlap=60, chunk_id=4377))

In [155]:
get_neo4j_docs("Wat moet ik doen bij nierstenen?")[0]

{'scrape_dt': '12/11/2023 15:26:50', 'webpage_id': 1, 'title': '24 uur ph-metrie meting', 'content': "\nWe weten dat reflux en opboeren voor een aantal klachten verantwoordelijk kunnen zijn. Via de impedantiemeting onderzoeken we of je klachten door reflux komen. Er zijn bovendien nog een aantal zeldzamere stoornissen die we met deze techniek kunnen opsporen.\n\n\n\n## Hoe gebeurt de meting?\n\nDe impedantiemeting gebeurt door een heel fijn buisje (2 mm), dat we via de neus tot in de slokdarm brengen. De sonde is verbonden met een klein computertje dat alle gegevens bewaart. De sonde moet ongeveer 24 uur ter plaatse blijven. We brengen ze in het ziekenhuis in en starten de meting terwijl je bij ons bent.\n\n\n\n## Nuchter zijn\n\n### Onderzoek voor 12 uur\n\n* Je mag na middernacht niets meer eten.\n* Je mag na middernacht niet meer roken.\n* Tot 2 uur voor het onderzoek mag je heldere vloeistof drinken. Nadien mag je niet meer drinken.\n\n### Onderzoek na 12 uur\n\n* Voor 8 uur ‘s och

Document(page_content='## Wat moet je doen?\n\nOm zoveel mogelijk informatie uit het onderzoek te krijgen, hebben we jouw medewerking nodig. Hierop moet je letten:', metadata=ChunkMetadata(chunk_size=500, embedding_model='hkunlp/instructor-xl', chunk_order=12, chunk_overlap=60, chunk_id=12))

In [172]:
neo4j_qa_retriever.get_relevant_documents(
    "Wat moet ik doen bij nierstenen?"
)

[Document(page_content='## Wat moet je doen?\n\nOm zoveel mogelijk informatie uit het onderzoek te krijgen, hebben we jouw medewerking nodig. Hierop moet je letten:', metadata={'chunk_size': 500, 'retrieval_embedding_model': 'hkunlp/instructor-xl with instruction: Represent the Medical paragraph for retrieval', 'retrieval_embedding': [0.048542533069849014, 0.02063559927046299, 0.007370846811681986, -0.045037396252155304, -0.03728263080120087, -0.05444404110312462, -0.06552710384130478, 0.01671726256608963, -0.005763005930930376, -0.051513928920030594, 0.004760378040373325, -0.003776076016947627, -0.05145992338657379, -0.1214095875620842, -0.011273390613496304, -0.008379826322197914, 0.01148592121899128, -0.02809065766632557, 0.011647213250398636, 0.012349169701337814, 0.0033375974744558334, 0.03657405823469162, -0.022929349914193153, 0.038133520632982254, -0.0008952018106356263, -0.07329059392213821, 0.011126615107059479, 0.0044308812357485294, 0.0012116250582039356, 0.0173510797321796

In [173]:
neo4j_retrieval_retriever.get_relevant_documents(
    "Wat moet ik doen bij nierstenen?"
)

Failed to read from defunct connection IPv4Address(('20.111.32.34', 7687)) (ResolvedIPv4Address(('20.111.32.34', 7687)))


ServiceUnavailable: Failed to read from defunct connection IPv4Address(('20.111.32.34', 7687)) (ResolvedIPv4Address(('20.111.32.34', 7687)))

In [150]:
import textwrap
from langchain.chains import LLMChain

BASE_PROMPT_TEMPLATE_NL = """
JE BENT EEN ZIEKENHUISADMINISTRATEUR.
BEANTWOORD DE VRAAG DIE VOLGT ALLEEN OP BASIS VAN DE ONDERSTAANDE CONTEXT:
{context}

VRAAG: 
{question}

INSTRUCTIES BIJ HET ANTWOORDEN:
- ALGEMEEN:
    - GEEF DUIDELIJK AAN ALS HET ANTWOORD NIET GEVONDEN KAN WORDEN IN DE DATABASE
    - ANTWOORD ALTIJD IN HET {language}
- INDIEN JE NIETS VINDT IN DE CONTEXT, VERMELD DAN:
    - LIJST DE MEEST GELIJKE CONTEXT OP MET KOMMA'S
    - DE URL VAN DE CONTEXT
- INDIEN JE WEL IETS VINDT IN DE CONTEXT, VERMELD DAN:
    - LIJST DE RELEVANTE CONTEXT OP MET KOMMA'S
    - DE BRON (URL) VAN DE CONTEXT.
"""

# QUESTION =============================================
question = "Wat moet ik doen met"

## RETRIEVE DOCUMENTS =================
# Get docs from Neo4j
docs: List[langchain.schema.document.Document] = (
    # neo4j_retriever.get_relevant_documents(
    #     question
    # )
    get_neo4j_docs(question)
)

llm_prompt = PromptTemplate(
    input_variables=["context", "question", "language"],
    # partial_variables={"Customer_Name", "Customer_State", "Customer_Gender"},
    template=BASE_PROMPT_TEMPLATE_NL,
)

llm = ChatOpenAI(temperature=1)
llm_chain = LLMChain(
    prompt=llm_prompt, 
    llm=llm,
    verbose=True
)

llm_answer = llm_chain.invoke(
    input={
        "question": question,
        "context": docs_to_str(docs), 
        "language": "Nederlands"
    },
    return_only_outputs=True,
    include_run_info=False
)

# Show output
# wrapped_context = textwrap.wrap(llm_prompt.)

# Show answer 
wrapped_output = textwrap.wrap(llm_answer['text'], width=80)
for line in wrapped_output:
    print(line)

AttributeError: 'ChunkMetadata' object has no attribute 'items'

In [76]:
print(docs_to_str(docs))

========================= Document 1 =========================
=============== Document 1 Metadata ===============
- chunk_size: 500 
- embedding_model: hkunlp/instructor-xl 
- chunk_order: 2431 
- chunk_overlap: 60 
- chunk_id: 2431 
=============== Document 1's Content===============
Er zijn verschillende types nierstenen (zes om precies te zijn), met elk hun eigen oorzaak en ontstaansmechanisme. Als je het type steen en de risicofactoren kent, kun je behandeling daarop afstemmen en is advies op maat mogelijk. 


Heel concreet geeft het metabool bilan een antwoord op de volgende vragen:


* Waarom maak ik nierstenen aan?
* Welke maatregelen zijn nodig om mijn kans op herval te verminderen.



## Voor wie?
========================= Document 2 =========================
=============== Document 2 Metadata ===============
- chunk_size: 500 
- embedding_model: hkunlp/instructor-xl 
- chunk_order: 3862 
- chunk_overlap: 60 
- chunk_id: 3862 
=============== Document 2's Content============